In [ ]:
"""
Purpose: Develop the tools to be able to restrict a concept network using pandas evals


"""

In [1]:
from os import sys
sys.path.append("../../meshAfterParty/meshAfterParty/")
from importlib import reload

In [2]:
import pandas_utils as pu
import pandas as pd
from pathlib import Path

# Example Concept Network

In [3]:
compressed_neuron_path = Path("../test_neurons/test_objects/12345_2_soma_practice_decompress")

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)

import system_utils as su

with su.suppress_stdout_stderr():
    recovered_neuron = nru.decompress_neuron(filepath=compressed_neuron_path,
                      original_mesh=compressed_neuron_path)

recovered_neuron

# Turning a concept_network into a dataframe

In [ ]:
import networkx as nx
nx.draw(recovered_neuron.concept_network)

In [ ]:
"""
How to turn a concept map into a pandas table with only the limb_idx and node_idx
"""
def convert_neuron_to_branches_dataframe(curr_neuron):
    limb_idxs = curr_neuron.get_limb_node_names()
    limb_node_idx_dicts = []

    for l in limb_idxs:
        limb_node_idx_dicts += [dict(limb=l,node=int(k)) for k in 
                                curr_neuron.concept_network.nodes[l]["data"].concept_network.nodes()]

    df = pd.DataFrame(limb_node_idx_dicts)
    return df

neuron_df = convert_neuron_to_branches_dataframe(curr_neuron = recovered_neuron)
neuron_df

# Functions that can be applied over each limb

In [ ]:
class run_options:
    def __init__(self,run_type="Limb"):
        self.run_type = run_type

    def __call__(self, f):
        f.run_type = self.run_type
        return f

In [ ]:
dir(recovered_neuron.concept_network.nodes["L0"]["data"].concept_network.nodes[1]["data"])

In [ ]:
"""
Want to have functions that just operate off of branch characteristics or limb characteristics
"""

#Branch Functions
import skeleton_utils as sk

@run_options(run_type="Branch")
def n_faces_branch(curr_branch):
    return len(curr_branch.mesh.faces)

@run_options(run_type="Branch")
def width(curr_branch):
    return curr_branch.width

@run_options(run_type="Branch")
def skeleton_distance_branch(curr_branch):
    try:
        #print(f"curr_branch.skeleton = {curr_branch.skeleton.shape}")
        return sk.calculate_skeleton_distance(curr_branch.skeleton)
    except:
        print(f"curr_branch.skeleton = {curr_branch.skeleton}")
        raise Exception("")


In [ ]:
node_array = np.array(recovered_neuron.concept_network.nodes["L1"]["data"].concept_network.nodes())


In [ ]:
#Limb Functions
nru = reload(nru)
sk = reload(sk)

"""
Rule: For the limb functions will either return
1) 1 number
2) 1 True/False Value
3) Array of 1 or 2 that matches the number of branches
4) a list of nodes that it applies to

"""
@run_options(run_type="Limb")
def skeleton_distance_limb(curr_limb):
    curr_skeleton = curr_limb.get_skeleton()
    return sk.calculate_skeleton_distance(curr_skeleton)

@run_options(run_type="Limb")
def n_faces_limb(curr_limb):
    return len(curr_limb.mesh.faces)

@run_options(run_type="Limb")
def merge_limbs(curr_limb):
    return "MergeError" in curr_limb.labels

@run_options(run_type="Limb")
def limb_error_branches(curr_limb):
    error_nodes = nru.classify_endpoint_error_branches_from_limb_concept_network(curr_limb.concept_network)
    node_names = np.array(list(curr_limb.concept_network.nodes()))
    return dict([(k,k in error_nodes) for k in node_names])

In [ ]:
recovered_neuron.get_limb_node_names()

# How to get an array for all branches based on the one function and the neuron object

In [ ]:
"""
Pseudocde: 
1) Determine if the function should be applied to a branch or a limb
2a) If Branch
- then just iterate over all branches, all limbs and compute and store value
3a) If Limb: 
- for each limb:
  i) run the function and recieve the returned result
  2) If only a single value is returned --> make dict[limb_idx][node_idx] = value all with the same value
  3) if dictionary of values: 
     a. check that keys match the node_names
     b. make dict[limb_idx][node_idx] = value for all nodes using the dictionary

Output: dictionary map[limb_idx][node_idx] = value

"""

nru = reload(nru)

def apply_function_to_neuron(current_neuron,current_function):
    function_mapping = dict([(limb_name,dict()) for limb_name in current_neuron.get_limb_node_names()])
    #if it was a branch function that was passed
    if current_function.run_type=="Branch":
        for limb_name in function_mapping.keys():
            curr_concept_network = current_neuron.concept_network.nodes[limb_name]["data"].concept_network
            for branch_idx in curr_concept_network.nodes():
                function_mapping[limb_name][branch_idx] = current_function(curr_concept_network.nodes[branch_idx]["data"])
    elif current_function.run_type=="Limb":
        #if it was a limb function that was passed
        """
        - for each limb:
          i) run the function and recieve the returned result
          2) If only a single value is returned --> make dict[limb_idx][node_idx] = value all with the same value
          3) if dictionary of values: 
             a. check that keys match the node_names
             b. make dict[limb_idx][node_idx] = value for all nodes using the dictionary
        
        """
        for limb_name in function_mapping.keys():
            function_return = current_function(current_neuron.concept_network.nodes[limb_name]["data"])
            curr_concept_network = current_neuron.concept_network.nodes[limb_name]["data"].concept_network
            if np.isscalar(function_return):
                for branch_idx in curr_concept_network.nodes():
                    function_mapping[limb_name][branch_idx] = function_return
            elif set(list(function_return.keys())) == set(list(curr_concept_network.nodes())):
                function_mapping[limb_name] = function_return
            else:
                raise Exception("The value returned from limb function was not a scalar nor did it match the keys of the limb branches")  
        
    else:
        raise Exception("Function recieved was neither a Branch nor a Limb")
        
    return function_mapping

curr_function =width
curr_function_mapping = apply_function_to_neuron(recovered_neuron,curr_function)
len(curr_function_mapping)

In [ ]:
import pandas_utils as pu
pu = reload(pu)
map_new_limb_node_value(neuron_df,curr_function_mapping,value_name=curr_function.__name__)
pu.surpress_scientific_notation()
print(f"Number of nans = {pu.n_nans_total(neuron_df)}")
#neuron_df[neuron_df["limb_error_branches"]==True]
neuron_df

# Creating the function that will make a dataframe with all of the functions specified (later will use the restriction)

In [ ]:
import numpy_utils as nu
import pandas_utils as pu

import neuron_searching as ns
ns = reload(ns)


returned_df = ns.generate_neuron_dataframe(recovered_neuron,functions_list=[
ns.n_faces_branch,
ns.width,
ns.skeleton_distance_branch,
ns.skeleton_distance_limb,
ns.n_faces_limb,
ns.merge_limbs,
ns.limb_error_branches
])

returned_df[returned_df["merge_limbs"] == True

# Exploring Pandas and Dataframe

In [ ]:
"""
Things need to figure out about pandas dataframe:
1) Creating new column witih data and without data
2) dataframe.eval and how works with:
- ands/or
- == True/False
- apply a dictionary to variable names so can do so programmatically restrict
- group by (so can get the limb/nodes grouped together)

"""

In [ ]:
neuron_df

# ----- Part 2: Starting the Work on How the Query Will work -----

In [45]:
"""

Arguments: 
1) concept network
2) qurey (string or list of dicts)
3) Extra Functions to run that generates: True/False/Numeric 
for every limb in the network
- List of functions or keywords (that map to specific functions)

Process: 
0) Generate a pandas table that originally has the limb index and node index
as a unique row to correspond to all the unique branches
1) Running the Limb functions
a. Convert all of the keywords into the corresponding functions that get the value
b. For each function in the list
    Create a new column for the function
   i) For each limb: 
       1) Run on whole limb and store the values in the dataframe (may need to do this at end)
       Look for any NaN values and either:
       a. Error
       b. Raise Warning but don't error
       
2) Processing of Query: 
- If recieved as string then don't need to process any
- If recieved as list of dictionaries --> turn into a string query

3) Apply the string query to the dataframe using df.eval

4) Return values:
a. All (limb_index,node_index)
b. Node names grouped in list under same limb index
   dict{ limb_idx:[node_idxs...],
         limb_idx:[node_idxs...]
   
   }
c. Data table used




"""



In [53]:
recovered_neuron.concept_network.nodes()

NodeView(('S0', 'L0', 'L1', 'L2', 'L3', 'L5', 'L10', 'S1', 'L4', 'L6', 'L7', 'L8', 'L9'))

In [57]:
nru.return_concept_network(recovered_neuron).nodes["L1"]

{'data': <neuron.Limb at 0x7f878f13b5c0>}

In [95]:
ns = reload(ns)
nru = reload(nru)

list_of_faces = [1038,5763,7063,11405]
branch_threshold = 60000
current_query = "n_faces_branch in @list_of_faces or skeleton_distance_branch > @branch_threshold"
local_dict=dict(list_of_faces=list_of_faces,branch_threshold=branch_threshold)


functions_list=[
ns.n_faces_branch,
"width",
ns.skeleton_distance_branch,
# ns.skeleton_distance_limb,
# "n_faces_limb",
# ns.merge_limbs,
# ns.limb_error_branches
]

returned_output = ns.query_neuron(recovered_neuron,
                         functions_list,
                          current_query,
                          local_dict=local_dict,
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=True)

Extracting concept network from neuron object
functions_list = [<function n_faces_branch at 0x7f877ef0dea0>, 'width', <function skeleton_distance_branch at 0x7f877ec2fe18>]
final_feature_functions = [<function n_faces_branch at 0x7f877ef0dea0>, <function width at 0x7f877c3692f0>, <function skeleton_distance_branch at 0x7f877ec2fe18>]


# ---- Part 2 Prework: Check what can do with df.eval and df.query -----------#

In [73]:
np.array_equal(np.sort(returned_dataF.query("limb==@curr_limb",local_dict=dict(curr_limb='L0'))["node"].to_numpy()),returned_output["L0"])

True

In [43]:
"""
Things want to check: 
1) Passing in variables with local_dict (yes it does!)

"""
import numpy as np

list_of_faces = [1038,5763,7063,11405]
branch_threshold = 31000
filtered_df = returned_df.query("n_faces_branch in @list_of_faces or skeleton_distance_branch > @branch_threshold",
                  local_dict=dict(list_of_faces=list_of_faces,branch_threshold=branch_threshold))

limb_branch_pairings = filtered_df[["limb","node"]].to_numpy()
np.unique(limb_branch_pairings[:,0])
limb_to_branch = dict([(k,np.sort(limb_branch_pairings[:,1][np.where(limb_branch_pairings[:,0]==k)[0]]).astype("int")) 
                       for k in np.unique(limb_branch_pairings[:,0])])
limb_to_branch

{'L0': array([ 0,  9, 10, 12, 14, 17, 18, 19, 21, 23, 27, 39, 45, 50, 56, 59, 60,
        65, 68, 70]),
 'L1': array([ 0,  2,  3,  4,  9, 10, 11, 14, 15, 20, 33, 35, 48]),
 'L2': array([ 0,  1,  5,  9, 10, 12, 14, 19, 29]),
 'L3': array([ 0,  1,  2,  4,  5, 22, 34]),
 'L4': array([ 0,  2,  3,  9, 13, 16, 18, 32]),
 'L5': array([ 0,  1,  3,  5, 14]),
 'L6': array([ 0,  1,  2,  7, 15]),
 'L7': array([ 0,  4,  7,  8, 10]),
 'L8': array([4, 5]),
 'L9': array([1])}

In [5]:
import numpy_utils as nu
import pandas_utils as pu

import neuron_searching as ns
ns = reload(ns)


returned_df = ns.generate_neuron_dataframe(recovered_neuron,functions_list=[
ns.n_faces_branch,
ns.width,
ns.skeleton_distance_branch,
ns.skeleton_distance_limb,
ns.n_faces_limb,
ns.merge_limbs,
ns.limb_error_branches
])

returned_df[returned_df["merge_limbs"] == True]

only one skeleton so no stacking needed


,limb,node,n_faces_branch,width,skeleton_distance_branch,skeleton_distance_limb,n_faces_limb,merge_limbs,limb_error_branches
73,L1,11,5763,1919.382958,23724.514761,1.176599e+06,176175,True,False
74,L1,14,7063,1791.959738,31024.460288,1.176599e+06,176175,True,False
75,L1,15,11405,1711.838444,52246.083671,1.176599e+06,176175,True,False
76,L1,20,1038,460.219090,16913.487597,1.176599e+06,176175,True,False
77,L1,49,62,1668.789583,430.349858,1.176599e+06,176175,True,False
78,L1,21,699,326.726151,19725.807405,1.176599e+06,176175,True,False
79,L1,30,1920,2173.483893,7233.961140,1.176599e+06,176175,True,False
80,L1,22,142,306.704363,5138.974176,1.176599e+06,176175,True,False
81,L1,23,1730,1062.909909,9781.700415,1.176599e+06,176175,True,False
82,L1,17,5209,1546.301408,26044.464366,1.176599e+06,176175,True,False


# Final Example of how query works: 


In [97]:
from os import sys
sys.path.append("../../meshAfterParty/meshAfterParty/")
from importlib import reload

import pandas_utils as pu
import pandas as pd
from pathlib import Path


compressed_neuron_path = Path("../test_neurons/test_objects/12345_2_soma_practice_decompress")

import neuron_utils as nru
nru = reload(nru)
import neuron
neuron=reload(neuron)

import system_utils as su

with su.suppress_stdout_stderr():
    recovered_neuron = nru.decompress_neuron(filepath=compressed_neuron_path,
                      original_mesh=compressed_neuron_path)

recovered_neuron

In [106]:
sk.graph_skeleton_and_mesh(other_meshes=[recovered_neuron.mesh])

In [111]:
recovered_neuron.plot_limb_concept_network(limb_idx=1,node_size=1,
                                          arrow_size=1)

In [121]:
ns = reload(ns)
ns.average_branch_length(recovered_neuron.concept_network.nodes["L1"]["data"])

22199.980722726454

In [126]:
import skeleton_utils as sk
sk = reload(sk)
ns = reload(ns)
nru = reload(nru)

list_of_faces = [1038,5763,7063,11405]
branch_threshold = 31000
#current_query = "n_faces_branch in @list_of_faces or skeleton_distance_branch > @branch_threshold or width > 10000"

local_dict=dict(list_of_faces=list_of_faces,branch_threshold=branch_threshold)


functions_list=[
ns.n_faces_branch,
"width",
ns.skeleton_distance_branch,
ns.skeleton_distance_limb,
"n_faces_limb",
ns.merge_limbs,
ns.limb_error_branches,
ns.average_branch_length
    
]

current_query = "(average_branch_length > 10000) and width > 1000"
current_query = "(n_faces_branch > 1000) and (width > 1000)"

returned_output = ns.query_neuron(recovered_neuron,
                         functions_list,
                          current_query,
                          local_dict=local_dict,
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)

returned_output

{'L0': array([ 0,  2,  9, 12, 17, 18, 19, 20, 21, 22, 23, 25, 28, 30, 32, 34, 38,
        39, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 59,
        60, 61, 62, 63, 65, 67, 69, 71]),
 'L1': array([ 2,  4,  7,  9, 10, 11, 12, 14, 15, 17, 23, 28, 30, 31, 33, 34, 35,
        37, 40, 42, 45]),
 'L10': array([0]),
 'L2': array([ 0,  1,  2,  5,  6,  8, 10, 11, 14, 16, 24, 34, 44, 46, 48]),
 'L3': array([ 0,  1,  3,  5,  7, 12, 19, 20, 23, 24, 26, 31, 34]),
 'L4': array([ 0,  4,  5,  9, 10, 13, 16, 18, 19, 20]),
 'L5': array([ 0,  1,  3,  5,  7, 11, 12, 13, 14, 15]),
 'L6': array([ 1,  2,  5,  7,  9, 10, 13, 15]),
 'L7': array([ 0,  1,  2,  4,  7,  8, 10, 17]),
 'L8': array([4, 5, 9])}

In [127]:
returned_output = ns.query_neuron(recovered_neuron,
                         functions_list,
                          current_query,
                          local_dict=local_dict,
                          return_dataframe=True,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)
returned_output

,limb,node,n_faces_branch,width,skeleton_distance_branch,skeleton_distance_limb,n_faces_limb,merge_limbs,limb_error_branches,average_branch_length
0,L0,46,2505,3771.240134,16544.280033,1.712813e+06,308343,False,False,23463.189617
1,L0,28,3980,1180.009647,18386.469136,1.712813e+06,308343,False,False,23463.189617
2,L0,47,7250,3011.199162,11666.610807,1.712813e+06,308343,False,False,23463.189617
3,L0,21,9530,1584.967602,38458.807931,1.712813e+06,308343,False,False,23463.189617
5,L0,53,3347,2283.324072,5976.299947,1.712813e+06,308343,False,False,23463.189617
...,...,...,...,...,...,...,...,...,...,...
293,L7,10,5313,1298.818387,35237.794341,4.396150e+05,58543,False,False,17584.601876
299,L7,17,3432,1359.583790,21648.782990,4.396150e+05,58543,False,False,17584.601876
308,L8,9,4149,1426.851819,23279.963128,2.567327e+05,29598,False,False,23339.336420
309,L8,5,8554,1015.467877,60922.702261,2.567327e+05,29598,False,False,23339.336420


In [ ]:
"""
Applications: 
1) Spine Detection
2) Recommended splits
- the width suddenly changes 
- angle of connection (with neighbors)
- arrows in opposite direection
- merge really close branch points 
     Look for high degree
- width discontinuities
     


Axon to Axon


Goal: 
Apply or Decline
Default functions --> importance (change those )
print out: why chose?

could go through them one at a time 


could get the line of points between two merged branches
line A 
line B


Workflow: 
1) Apply filters
2) Visualize
3) select the correct cut points (like control find and accept)
4) of all accept: link that has the points
4) apply the cuts in the length
"""